# ONNX Export Demo

Reference: https://pytorch.org/tutorials//beginner/onnx/export_simple_model_to_onnx_tutorial.html#save-the-onnx-model-in-a-file

## Train a simple linear regression model

In [1]:
import tensorweaver as torch
from tensorweaver.toy_datasets.get_celsius_fahrenheit_dataset import (
    get_celsius_fahrenheit_dataset,
)

In [2]:
# load the dataset
x_array, y_array = get_celsius_fahrenheit_dataset()

In [3]:
# define our model, a simple linear regression model
class LinearRegressionModel(torch.nn.Module):

    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1)  # One in and one out

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred


# create our model
our_model = LinearRegressionModel()

# convert the dataset to tensors
x = torch.tensor(x_array)  # temperature in celsius
y = torch.tensor(y_array)  # temperature in fahrenheit

# define our loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(our_model.parameters(), lr=0.0015)

# train our model
loss_path = []

for epoch in range(10000):
    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = our_model(x)

    # Compute and print loss
    loss = criterion(pred_y, y)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss_path.append(loss.item())
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 4844.748521018383
epoch 1000, loss 6.175406086208859
epoch 2000, loss 0.8484577318163624
epoch 3000, loss 0.8151147321064127
epoch 4000, loss 0.814906028068809
epoch 5000, loss 0.8149047217263782
epoch 6000, loss 0.8149047135495807
epoch 7000, loss 0.8149047134984003
epoch 8000, loss 0.8149047134980808
epoch 9000, loss 0.814904713498079


## Export our trained model to ONNX format

In [4]:
example_inputs = (torch.tensor(((0.0,), (100.0,))),)
onnx_program = torch.onnx.export(our_model, example_inputs)

In [5]:
onnx_program.save("model.onnx")

## Display our exported ONNX model by using netron

In [10]:
%pip install -q netron

import netron
address = netron.serve('model.onnx', verbosity=0)
netron.widget(address, height=800)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Using onnxruntime to load and run our exported ONNX model

In [6]:
%pip install -q onnxruntime

import onnxruntime

onnx_inputs = [tensor.numpy() for tensor in example_inputs]
print(f"Sample input: {onnx_inputs}")

 # will solve wired errors on some machines: pthread_setaffinity_np
session_options = onnxruntime.SessionOptions()
session_options.intra_op_num_threads = 1

ort_session = onnxruntime.InferenceSession(
    "./model.onnx",
    providers=["CPUExecutionProvider"],
    sess_options=session_options
)

onnxruntime_input = {input_arg.name: input_value for input_arg, input_value in zip(ort_session.get_inputs(), onnx_inputs)}

# ONNX Runtime returns a list of outputs
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)[0]

print("onnx runtime outputs: ", onnxruntime_outputs)
print("expected outputs: 0 celsius -> 32 fahrenheit, 100 celsius -> 212 fahrenheit")

Input length: 1
Sample input: [array([[  0.],
       [100.]])]
onnx runtime outputs:  [[ 31.8789119 ]
 [212.05132774]]
expected outputs: 0 celsius -> 32 fahrenheit, 100 celsius -> 212 fahrenheit
